In [1]:
import psycopg2
import json
import geopandas as gpd
import pandas as pd

# ✅ Database Connection Using psycopg2
POSTGIS_CONN = {
    "dbname": "crowdcounting",
    "user": "admin",
    "password": "admin",
    "host": "40.84.231.179",
    "port": "5434",
}

In [2]:
roi = gpd.read_file('playgroundROI.gpkg')
roi.explore()

In [5]:
import geopandas as gpd
import pandas as pd
import psycopg2
from shapely.ops import unary_union

# 1) Carga ROI y asegúrate de EPSG:4326
roi = gpd.read_file('playgroundROI.gpkg')
if roi.crs is None:
    raise ValueError("El GeoPackage no tiene CRS. Asigna uno correcto o reproyecta.")
if roi.crs.to_epsg() != 4326:
    roi = roi.to_crs(epsg=4326)

# Un solo polígono para la consulta (une si hay varios)
roi_geom = unary_union(roi.geometry)
roi_wkt = roi_geom.wkt  # lo enviaremos como parámetro WKT

# 2) Consulta con sjoin (ST_Intersects) + LIMIT 1000 + conteo total con ventana
SQL_QUERY = """
WITH roi as (
  SELECT ST_GeomFromText(%s, 4326) AS geom
),
filtered as (
  SELECT p.*
  FROM person_observed p
  JOIN roi r
    ON ST_Intersects(p.geom, r.geom)
)
SELECT
  *,
  COUNT(*) OVER() AS total_matches
FROM filtered
ORDER BY timestamp
"""

try:
    conn = psycopg2.connect(**POSTGIS_CONN)
    df = pd.read_sql(SQL_QUERY, conn, params=[roi_wkt])
    conn.close()
except Exception as e:
    print(f"Error connecting to database: {e}")
    df = pd.DataFrame()

# 3) Mostrar resultados y total
if not df.empty:
    total = int(df.loc[0, 'total_matches'])
    print(f"Total dentro del ROI: {total}")
    print(f"Mostrando {len(df)} filas (máximo 1000).")
else:
    print("No hubo coincidencias dentro del ROI o la consulta falló.")


Total dentro del ROI: 1592490
Mostrando 1592490 filas (máximo 1000).


In [6]:
df

,id,id_person,lat,long,timestamp,geom,camera_name,coordinate_x,coordinate_y,tracklet_id,total_matches
0,columpioscam3-2024-09-25_062223.mp4:b17e1ad8-0...,b17e1ad8-0c2c-44e4-ba72-54c2dd3881a4,25.653059,-100.286003,2024-09-25 12:22:23.666686+00:00,0101000020E6100000BCFFFFDF4D1259C0AAFFFFDF2EA7...,None,NaN,NaN,None,1592490
1,columpioscam3-2024-09-25_062223.mp4:b17e1ad8-0...,b17e1ad8-0c2c-44e4-ba72-54c2dd3881a4,25.653059,-100.286003,2024-09-25 12:22:23.708353+00:00,0101000020E6100000BCFFFFDF4D1259C0AAFFFFDF2EA7...,None,NaN,NaN,None,1592490
2,columpioscam3-2024-09-25_062223.mp4:b17e1ad8-0...,b17e1ad8-0c2c-44e4-ba72-54c2dd3881a4,25.653057,-100.286003,2024-09-25 12:22:23.750021+00:00,0101000020E6100000BCFFFFDF4D1259C043FFFFBF2EA7...,None,NaN,NaN,None,1592490
3,columpioscam3-2024-09-25_062223.mp4:b17e1ad8-0...,b17e1ad8-0c2c-44e4-ba72-54c2dd3881a4,25.653057,-100.286003,2024-09-25 12:22:23.791689+00:00,0101000020E6100000BCFFFFDF4D1259C043FFFFBF2EA7...,None,NaN,NaN,None,1592490
4,columpioscam3-2024-09-25_062223.mp4:b17e1ad8-0...,b17e1ad8-0c2c-44e4-ba72-54c2dd3881a4,25.653055,-100.286003,2024-09-25 12:22:23.833357+00:00,0101000020E6100000BCFFFFDF4D1259C0F5FFFF9F2EA7...,None,NaN,NaN,None,1592490
...,...,...,...,...,...,...,...,...,...,...,...
1592485,columpioscam1-2025-01-07_21-57-51.mp4:ccb84efc...,ccb84efc-0090-468d-a288-dad57c51e9ca,25.653204,-100.285637,2025-01-08 03:57:52.995031+00:00,0101000020E6100000000000E0471259C00000006038A7...,urn:ngsi-ld:camera:columpiosCam1,0.298534,0.093367,columpioscam1-2025-01-07_21-57-51_1_48_2565320...,1592490
1592486,columpioscam1-2025-01-07_21-57-51.mp4:488b4e47...,488b4e47-64e4-4832-a7e8-2ccf43d49ad6,25.653206,-100.285645,2025-01-08 03:57:53.660041+00:00,0101000020E610000000000000481259C00000008038A7...,urn:ngsi-ld:camera:columpiosCam1,0.277877,0.067316,columpioscam1-2025-01-07_21-57-51_64_67_256532...,1592490
1592487,columpioscam1-2025-01-07_21-57-51.mp4:488b4e47...,488b4e47-64e4-4832-a7e8-2ccf43d49ad6,25.653206,-100.285645,2025-01-08 03:57:53.701604+00:00,0101000020E610000000000000481259C00000008038A7...,urn:ngsi-ld:camera:columpiosCam1,0.277535,0.066065,columpioscam1-2025-01-07_21-57-51_64_67_256532...,1592490
1592488,columpioscam1-2025-01-07_21-57-51.mp4:488b4e47...,488b4e47-64e4-4832-a7e8-2ccf43d49ad6,25.653206,-100.285645,2025-01-08 03:57:53.743167+00:00,0101000020E610000000000000481259C00000008038A7...,urn:ngsi-ld:camera:columpiosCam1,0.276832,0.065206,columpioscam1-2025-01-07_21-57-51_64_67_256532...,1592490


In [24]:
import os
from datetime import datetime
from azure.storage.blob import BlobServiceClient
from tqdm import tqdm

account_url = 'https://cienciaciudades2024.blob.core.windows.net'
container = 'crowdcounting'
sas_token = 'sp=racwdl&st=2025-02-05T16:48:09Z&se=2026-04-02T00:48:09Z&spr=https&sv=2022-11-02&sr=c&sig=pXOsajenPg9lvILk6OozoyE%2BUz%2FDXivaBWwIioLyRVo%3D'
blob_service_client = BlobServiceClient(account_url, credential=sas_token)
azure_path = df.iloc[-1]['id'].split(':')[0]
if isinstance(azure_path, str):
    azure_path = [azure_path]

container_client = blob_service_client.get_container_client(container)

download_paths = []

for path in azure_path:
    print(f"Downloading {path}...")
    blob_client = container_client.get_blob_client(path)
    download_file_path = 'Downloads/' + path
    os.makedirs(os.path.dirname(download_file_path), exist_ok=True)
    # Download the blob
    with open(download_file_path, "wb") as download_file:
        download_stream = blob_client.download_blob()
        download_file.write(download_stream.readall())
    download_paths.append(download_file_path)

